In [3]:
import pickle, os
import numpy as np
import pickle, joblib
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model, get_custom_objects
from tensorflow.keras.optimizers import SGD, Nadam, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import sigmoid
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras import backend as K
import gc
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2022-11-06 16:57:44.663275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 16:57:44.952464: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-06 16:57:50.751951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-06 16:57:50.752057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Coul

In [4]:
def show_train_history(train_history):
    fig=plt.gcf()
    fig.set_size_inches(16, 6)
    plt.subplot(121)
    plt.plot(train_history[0])
    plt.plot(train_history[1])
    plt.title("Train History")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(["train", "validation"], loc="upper left")
    plt.subplot(122)
    plt.plot(train_history[2])
    plt.plot(train_history[3])
    plt.title("Train History")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(["train", "validation"], loc="upper left")
    plt.close(fig)
    return fig
def optimizer_f(optimizer):
    if optimizer == "sgd":
        return SGD(learning_rate=0.001, momentum=0.9, decay=0, nesterov=True)
    if optimizer == "nadam":
        return Nadam(learning_rate=0.001)
    if optimizer == 'adam':
        return Adam(learning_rate=0.001)
#def swish(x, beta = 1):
#    return (x * sigmoid(beta * x))
#get_custom_objects().update({'swish': swish})
def transforms(p=0.75):
    return A.Compose([
        A.RandomToneCurve(scale=0.3, p=1),
        #A.CLAHE(p=0.5),
        #A.ColorJitter(brightness=1,contrast=0, saturation=0, hue=0),
        #A.RandomBrightnessContrast(),
        A.RandomGamma(p=1),
        A.Blur(blur_limit=3, p=1)
    ], p=p)
def augmentor(image):
    aug = transforms()
    aug_img = aug(image = np.array(image).astype('uint8'))['image']
    aug_img = preprocess_input(aug_img)
    return aug_img
def callbacks_list_fn(file, loss, save_weights_only_, earlystop = True):
    if loss=="sparse":
        monitor_acc = 'val_sparse_categorical_accuracy'
    else:
        monitor_acc = 'val_accuracy'
        
    lrate = ReduceLROnPlateau(monitor = monitor_acc, factor=0.75, patience=10, mode='max', min_delta=0.00001, 
                              cooldown=0, threshold_mode='rel', min_lr=1e-8)    
    ck_callback_acc = ModelCheckpoint(file+'/weights_acc.h5', monitor=monitor_acc, mode='max', 
                                       save_best_only=True, save_weights_only=save_weights_only_)
    ck_callback_los = ModelCheckpoint(file+'/weights_loss.h5', monitor='val_loss', mode='min', 
                                      save_best_only=True, save_weights_only=save_weights_only_)
    csv_callback = CSVLogger(file+'/log.csv', append=True, separator=',')
    
    if earlystop:
        earlyst = EarlyStopping(monitor="val_loss", patience = 40, mode='min', min_delta=0.00001)    
        return [ck_callback_acc, ck_callback_los, csv_callback, lrate, earlyst]
    else:
        return [ck_callback_acc, ck_callback_los, csv_callback, lrate]
def model_build(activation_, normal_, l2_, optimizer_, pooling_, image_type, loss_):
    # 預先訓練好的模型 -- Xception_dataset, 不含後三層(辨識層)
    # base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    # base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    # base_model.trainable = False
    if image_type=='merge':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel3':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel3_127':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel6':
        base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 6))
    if image_type=='chanel12':
        base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 12))
    if image_type=='multi':
        base_model_1 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
        base_model_2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
        # 連接自訂層
        for layer in base_model_1.layers:
            layer._name = layer._name + str("_1")
        for w in base_model_1.weights:
            w._handle_name = w.name + str("_1")
        for layer in base_model_2.layers:
            layer._name = 'EP_' + layer._name + str("_2")
        for w in base_model_2.weights:
            w._handle_name = 'EP_' + w.name + str("_2")
        base_model = layers.concatenate([base_model_1.output, base_model_2.output])
    
    if pooling_ == "max":
        if image_type=='multi':
            x = layers.GlobalMaxPooling2D()(base_model)
        else:
            x = layers.GlobalMaxPooling2D()(base_model.output)
    if pooling_ == "ave":
        if image_type=='multi':
            x = layers.GlobalAveragePooling2D()(base_model)
        else:
            x = layers.GlobalAveragePooling2D()(base_model.output)
    if pooling_ == "conv":
        if image_type=='multi':
            x = layers.Conv2D(1280, (7, 7), activation="relu")(base_model)
        else:
            x = layers.Conv2D(1280, (7, 7), activation="relu")(base_model.output)
    if pooling_ == 'all':
        x1 = layers.GlobalMaxPooling2D()(base_model.output)
        x2 = layers.GlobalAveragePooling2D()(base_model.output)
        x3 = layers.Conv2D(1280, (7, 7), activation="relu")(base_model.output)
        x3 = layers.BatchNormalization()(x3)
        x3 = layers.Activation(activation_)(x3)
        x3 = layers.Flatten()(x3)
        x = layers.concatenate([x1, x2, x3])
    
    if pooling_ == "conv":
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        x = layers.Flatten()(x)
    else:        
        if normal_ == 'drop':
            x = layers.Dropout(0.25)(x)
        if normal_ == 'batch':
            x = layers.BatchNormalization()(x)
            x = layers.Activation(activation_)(x)
        
    x = layers.Dense(1024, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)    
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(256, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(64, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(6, activation='softmax')(x)

    # 設定新模型的 inputs/outputs
    if image_type=='multi':
        model = Model(inputs=[base_model_1.input, base_model_2.input], outputs=x)
    else:
        model = Model(inputs=base_model.input, outputs=x)
    
    if loss_=="sparse":
        model.compile(optimizer = optimizer_f(optimizer_), 
                      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])    
    else:
        model.compile(optimizer = optimizer_f(optimizer_), 
                      #loss = "categorical_crossentropy", 
                      loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1),
                      metrics=["accuracy"])    
        
    return model

In [3]:
pkl_route = 'chanel3'
path_list = [os.path.join(dirpath,filename) for dirpath, _, filenames in os.walk('07_classification_training_' + pkl_route) for filename in filenames if 'weights_final.h5' in filename]
with open('06_classification_image/classification_dataset_' + pkl_route + '.pkl', 'rb') as f:
    dataset = pickle.load(f)
    
train_X_key = [x for x in dataset['training_X'].keys() if x.split("_")[1]=='0']
testing_X_key = [x for x in dataset['testing_X'].keys() if x.split("_")[1]=='0']

train_X = [dataset['training_X'][x]['image'] for x in train_X_key]
train_X = preprocess_input(np.array(train_X))
test_X = [dataset['testing_X'][x]['image'] for x in testing_X_key]
test_X = preprocess_input(np.array(test_X))
train_Y = pd.DataFrame(dataset['training_Y'])
train_Y['key'] = dataset['training_X'].keys()
train_Y = np.array(train_Y[train_Y['key'].str.contains("_0")][[0,1,2,3,4,5]])
test_Y = pd.DataFrame(dataset['testing_Y'])
test_Y['key'] = dataset['testing_X'].keys()
test_Y = np.array(test_Y[test_Y['key'].str.contains("_0")][[0,1,2,3,4,5]])
del dataset

In [5]:
result = []
result_groupby = []
result_acc = []
dic = []
for p_ in path_list:
    route = '/'.join(p_.split("/")[:-1])
    optimizer_ = route.split('/')[1]
    pool_ = route.split('/')[2]
    activation_, normal_, l2_ = route.split('/')[-1].split("_")
    model = model_build(activation_, normal_, l2_, optimizer_, pool_, 'merge', "sparse")
    
    for h5_ in ['weights_acc.h5','weights_final.h5','weights_loss.h5']:    
        model_route_tmp = route + "/" + h5_
        print(model_route_tmp)
        model.load_weights(model_route_tmp)
        train_X_ = model.predict(train_X)
        test_X_ = model.predict(test_X)
        train_X_ = pd.concat([pd.DataFrame({'file_name':list(train_X_key),
                                            'real_class':np.argmax(train_Y ,axis=1), 
                                            'predict_class':np.argmax(train_X_ ,axis=1),
                                            'dataset':'train'}), 
                              pd.DataFrame(train_X_)], axis=1)
        test_X_ = pd.concat([pd.DataFrame({'file_name':list(testing_X_key),
                                           'real_class':np.argmax(test_Y ,axis=1), 
                                           'predict_class':np.argmax(test_X_ ,axis=1),
                                           'dataset':'test'}),
                             pd.DataFrame(test_X_)], axis=1)
        df = pd.concat([train_X_, test_X_]).reset_index(drop = True)
        df['file'] = [x.split("_")[0] for x in df['file_name'].tolist()]

        df_groupby = df.groupby(['real_class','predict_class','dataset']).size().rename('num').reset_index(drop = False)
        df_groupby = pd.merge(df_groupby[df_groupby['real_class']==df_groupby['predict_class']],
                              df.groupby(['real_class','dataset']).size().rename('totla_num').reset_index(drop = False),
                              on = ['real_class','dataset'], how = "left")
        df_groupby['per_num'] = df_groupby['num']/df_groupby['totla_num']
        del df_groupby['predict_class']
        df_acc = df_groupby \
          .pivot_table(index=['real_class'],columns='dataset',values=['num','totla_num','per_num'],fill_value=0) \
          .reset_index(drop = False)
        df_acc.columns = df_acc.columns.map('_'.join).str.strip('_')
        
        dic_ = {'df_acc_test': sum(df_acc['num_test'])/sum(df_acc['totla_num_test']),
                'df_acc_train': sum(df_acc['num_train'])/sum(df_acc['totla_num_train']),
                'model_route': model_route_tmp}
        dic.append(dic_)

        df_groupby['model_route'] = model_route_tmp
        df_acc['model_route'] = model_route_tmp
        df['model_route'] = model_route_tmp

        result.append(df)
        result_groupby.append(df_groupby)
        result_acc.append(df_acc)
        
        tf.keras.backend.clear_session() 
        gc.collect()
        
    del model

result = pd.concat(result).reset_index(drop = True)
result_groupby = pd.concat(result_groupby).reset_index(drop = True)
result_acc = pd.concat(result_acc).reset_index(drop = True)
dic = pd.DataFrame(dic).reset_index(drop = True)

result.to_csv('07_classification_training_' + pkl_route + "/result.csv", index = False)
result_groupby.to_csv('07_classification_training_' + pkl_route + "/result_groupby.csv", index = False)
result_acc.to_csv('07_classification_training_' + pkl_route + "/result_acc.csv", index = False)
dic.to_csv('07_classification_training_' + pkl_route + "/result_all_acc.csv", index = False)

07_classification_training_chanel3/sgd/max/gelu_batch_0.1/weights_acc.h5
7/7 [==============================] - 1s 171ms/step
07_classification_training_chanel3/sgd/max/gelu_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 168ms/step
07_classification_training_chanel3/sgd/max/gelu_batch_0.1/weights_loss.h5
7/7 [==============================] - 1s 172ms/step
07_classification_training_chanel3/sgd/max/gelu_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 214ms/step
07_classification_training_chanel3/sgd/max/gelu_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 160ms/step
07_classification_training_chanel3/sgd/max/gelu_drop_0.1/weights_loss.h5
7/7 [==============================] - 1s 158ms/step
07_classification_training_chanel3/sgd/max/relu_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 217ms/step
07_classification_training_chanel3/sgd/max/relu_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 

7/7 [==============================] - 1s 233ms/step
07_classification_training_chanel3/sgd/conv/tanh_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 176ms/step
07_classification_training_chanel3/sgd/conv/tanh_batch_0.1/weights_loss.h5
7/7 [==============================] - 1s 171ms/step
07_classification_training_chanel3/sgd/ave/gelu_batch_0.1/weights_acc.h5
7/7 [==============================] - 1s 224ms/step
07_classification_training_chanel3/sgd/ave/gelu_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 171ms/step
07_classification_training_chanel3/sgd/ave/gelu_batch_0.1/weights_loss.h5
7/7 [==============================] - 1s 169ms/step
07_classification_training_chanel3/sgd/ave/gelu_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 210ms/step
07_classification_training_chanel3/sgd/ave/gelu_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 160ms/step
07_classification_training_chanel3/sgd/ave/gelu_drop_0.1/

7/7 [==============================] - 1s 170ms/step
07_classification_training_chanel3/adam/conv/swish_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 173ms/step
07_classification_training_chanel3/adam/conv/swish_batch_0.1/weights_loss.h5
7/7 [==============================] - 1s 172ms/step
07_classification_training_chanel3/adam/conv/relu_batch_0.1/weights_acc.h5
7/7 [==============================] - 1s 168ms/step
07_classification_training_chanel3/adam/conv/relu_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 169ms/step
07_classification_training_chanel3/adam/conv/relu_batch_0.1/weights_loss.h5
7/7 [==============================] - 1s 169ms/step
07_classification_training_chanel3/adam/conv/tanh_batch_0.1/weights_acc.h5
7/7 [==============================] - 1s 167ms/step
07_classification_training_chanel3/adam/conv/tanh_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 169ms/step
07_classification_training_chanel3/adam/c

7/7 [==============================] - 1s 181ms/step
07_classification_training_chanel3/nadam/conv/swish_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 162ms/step
07_classification_training_chanel3/nadam/conv/swish_drop_0.1/weights_loss.h5
7/7 [==============================] - 1s 161ms/step
07_classification_training_chanel3/nadam/conv/tanh_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 177ms/step
07_classification_training_chanel3/nadam/conv/tanh_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 159ms/step
07_classification_training_chanel3/nadam/conv/tanh_drop_0.1/weights_loss.h5
7/7 [==============================] - 1s 190ms/step
07_classification_training_chanel3/nadam/conv/swish_batch_0.1/weights_acc.h5
7/7 [==============================] - 1s 169ms/step
07_classification_training_chanel3/nadam/conv/swish_batch_0.1/weights_final.h5
7/7 [==============================] - 1s 204ms/step
07_classification_training_chanel3/na

7/7 [==============================] - 1s 163ms/step
07_classification_training_chanel3/nadam/all/relu_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 164ms/step
07_classification_training_chanel3/nadam/all/relu_drop_0.1/weights_loss.h5
7/7 [==============================] - 1s 164ms/step
07_classification_training_chanel3/nadam/all/swish_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 166ms/step
07_classification_training_chanel3/nadam/all/swish_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 165ms/step
07_classification_training_chanel3/nadam/all/swish_drop_0.1/weights_loss.h5
7/7 [==============================] - 1s 182ms/step
07_classification_training_chanel3/nadam/all/tanh_drop_0.1/weights_acc.h5
7/7 [==============================] - 1s 167ms/step
07_classification_training_chanel3/nadam/all/tanh_drop_0.1/weights_final.h5
7/7 [==============================] - 1s 168ms/step
07_classification_training_chanel3/nadam/all/ta

In [13]:
print(dic[dic['df_acc_test']>0.86]['model_route'].tolist())
url_ = '07_classification_training_chanel3/nadam/ave/tanh_drop_0.1/weights_loss.h5'
result_acc[result_acc['model_route']==url_]

['07_classification_training_chanel3/nadam/ave/tanh_drop_0.1/weights_loss.h5']


,real_class,num_test,num_train,per_num_test,per_num_train,totla_num_test,totla_num_train,model_route
1523,0,12,173,0.631579,1.000000,19,173,07_classification_training_chanel3/nadam/ave/t...
1524,1,26,309,0.764706,0.996774,34,310,07_classification_training_chanel3/nadam/ave/t...
1525,2,52,500,0.928571,1.000000,56,500,07_classification_training_chanel3/nadam/ave/t...
1526,3,42,436,0.875000,1.000000,48,436,07_classification_training_chanel3/nadam/ave/t...
1527,4,29,293,0.878788,1.000000,33,293,07_classification_training_chanel3/nadam/ave/t...
1528,5,20,194,1.000000,1.000000,20,194,07_classification_training_chanel3/nadam/ave/t...
